In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


1.[Reshape & pivot & melt](#Pivot&Stack)


   [pivot](#pivot)
   
   [pivot_table](#pivot_table)
   
  
   [melt](#melt)
   
   
   2.[working with text data](#series.str)
   
   [strip](#strip)
   
   [split](#split)
   
   [replace](#replace)
   
   3.[fillna](#fillna)

## Pivot&Stack


## 下面这个data他的index是有名字的，column也是有名字的！！！

In [2]:
data= pd.DataFrame(np.arange(6).reshape((2, 3)),index=pd.Index(['Ohio', 'Colorado'], name='state'), columns=pd.Index(['one', 'two', 'three'], name='number'))

In [3]:
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


## stack可以把列名，堆在row上

In [4]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

## 用unstack可以恢复。

In [5]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


## 可以给函数传入我们想要他unstack的level

In [6]:
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [7]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [8]:
result.unstack(1)

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [9]:
result.unstack('number')

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


## nan的情况

In [10]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])
data2 = pd.concat([s1, s2], keys=['one', 'two'])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [11]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [12]:
#stack回去，会自动赋值为0
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [13]:
#甚至可以增加一些row，通过dropna = False
data2.unstack().stack(dropna = False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

## 变化level的层次

## 先stack一下，把number放到row上，成为inner-most level

In [14]:
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [15]:
df = pd.DataFrame({'left': result, 'right': result + 5}, columns=pd.Index(['left', 'right'], name='side'))

In [16]:
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

## 然后unstack state，再stack state!

In [17]:
df.unstack(0)

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [18]:
df.unstack(0).stack('state')

side             left  right
number state                
one    Ohio         0      5
       Colorado     3      8
two    Ohio         1      6
       Colorado     4      9
three  Ohio         2      7
       Colorado     5     10

## pivot

In [19]:
df = pd.DataFrame({'foo': ['one','one','one','two','two','two'],
                       'bar': ['A', 'B', 'C', 'A', 'B', 'C'],
                       'baz': [1, 2, 3, 4, 5, 6]})
df

,foo,bar,baz
0,one,A,1
1,one,B,2
2,one,C,3
3,two,A,4
4,two,B,5
5,two,C,6


In [20]:
df.pivot(index = 'foo', columns = 'bar', values = 'baz')

bar,A,B,C
foo,,,
one,1,2,3
two,4,5,6


In [21]:
df.pivot(index = 'foo', columns = 'bar')

baz      
bar   A  B  C
foo          
one   1  2  3
two   4  5  6

In [22]:
df.pivot(index = 'foo', columns = 'bar')['baz']

bar,A,B,C
foo,,,
one,1,2,3
two,4,5,6


## 其实pivot是一个set_index然后unstack的过程的简化

In [23]:
df.set_index(['foo','bar'])

baz
foo bar     
one A      1
    B      2
    C      3
two A      4
    B      5
    C      6

In [24]:
df.set_index(['foo','bar']).unstack('bar')

baz      
bar   A  B  C
foo          
one   1  2  3
two   4  5  6

## pivot_table

In [25]:
df = pd.DataFrame({"A": ["foo", "foo", "foo", "foo", "foo",
...                          "bar", "bar", "bar", "bar"],
...                    "B": ["one", "one", "one", "two", "two",
...                          "one", "one", "two", "two"],
...                    "C": ["small", "large", "large", "small",
...                          "small", "large", "small", "small",
...                          "large"],
...                    "D": [1, 2, 2, 3, 3, 4, 5, 6, 7]})

In [26]:
df

,A,B,C,D
0,foo,one,small,1
1,foo,one,large,2
2,foo,one,large,2
3,foo,two,small,3
4,foo,two,small,3
5,bar,one,large,4
6,bar,one,small,5
7,bar,two,small,6
8,bar,two,large,7


In [27]:
table = pd.pivot_table(df, values = 'D', index = ['A','B'], columns = ['C'], aggfunc = np.sum)
table

C        large  small
A   B                
bar one    4.0    5.0
    two    7.0    6.0
foo one    4.0    1.0
    two    NaN    6.0

## unstack

In [28]:
index = pd.MultiIndex.from_tuples([('one', 'a'), ('one', 'b'),
...                                    ('two', 'a'), ('two', 'b')])

In [29]:
index

MultiIndex(levels=[['one', 'two'], ['a', 'b']],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [30]:
s = pd.Series(np.arange(1.0, 5.0), index=index)
s

one  a    1.0
     b    2.0
two  a    3.0
     b    4.0
dtype: float64

In [31]:
s.unstack(level=0)

,one,two
a,1.0,3.0
b,2.0,4.0


In [32]:
s.unstack(level=1)

,a,b
one,1.0,2.0
two,3.0,4.0


## 默认一般使用-1，因为这是最里面一层的index，即last level,一般会拿来作为新的column嘛！


In [33]:
s.unstack(level=-1)

,a,b
one,1.0,2.0
two,3.0,4.0


## stack

这个和pivot差不多，就是增加一个index，一个level作为row。

In [34]:
s = pd.DataFrame([[1,2],[3,4]], index = ['one','two'], columns = ['a','b'])

In [35]:
s

,a,b
one,1,2
two,3,4


In [36]:
s.stack()

one  a    1
     b    2
two  a    3
     b    4
dtype: int64

因为只有一个index，就默认stack到那里了，如果有多个，得注意用level = xx。

In [37]:
df

,A,B,C,D
0,foo,one,small,1
1,foo,one,large,2
2,foo,one,large,2
3,foo,two,small,3
4,foo,two,small,3
5,bar,one,large,4
6,bar,one,small,5
7,bar,two,small,6
8,bar,two,large,7


## melt

melt就像是unpivot。把原先累加好的东西打散。

In [38]:
df = pd.DataFrame({'A': {0: 'a', 1: 'b', 2: 'c'},
                    'B': {0: 1, 1: 3, 2: 5},                    'C': {0: 2, 1: 4, 2: 6}})

In [39]:
df

,A,B,C
0,a,1,2
1,b,3,4
2,c,5,6


In [40]:
pd.melt(df, id_vars = ['A'], value_vars = ['B'])

,A,variable,value
0,a,B,1
1,b,B,3
2,c,B,5


In [41]:
pd.melt(df, id_vars = ['A'], value_vars = ['B','C'])

,A,variable,value
0,a,B,1
1,b,B,3
2,c,B,5
3,a,C,2
4,b,C,4
5,c,C,6


还可以自定义新的column的名字

In [42]:
pd.melt(df, id_vars = ['A'], value_vars = ['B'], var_name = "myVarname", value_name = "myValue")

,A,myVarname,myValue
0,a,B,1
1,b,B,3
2,c,B,5


如果有多层index的column

In [43]:
df.columns = [list('ABC'),list('DEF')]

In [44]:
df

,A,B,C
,D,E,F
0,a,1,2
1,b,3,4
2,c,5,6


In [45]:
pd.melt(df, col_level = 0, id_vars = ['A'], value_vars = ['B'])

,A,variable,value
0,a,B,1
1,b,B,3
2,c,B,5


In [46]:
pd.melt(df, id_vars = [('A', 'D')], value_vars = [('B','E')])

,"(A, D)",variable_0,variable_1,value
0,a,B,E,1
1,b,B,E,3
2,c,B,E,5


## series.str

In [47]:
s = pd.Series(['A', 'B', 'C', 'Aaba', 'Baca', np.nan, 'CABA', 'dog', 'cat'])

In [48]:
s.str.upper()

0       A
1       B
2       C
3    AABA
4    BACA
5     NaN
6    CABA
7     DOG
8     CAT
dtype: object

In [49]:
s.str.len()

0    1.0
1    1.0
2    1.0
3    4.0
4    4.0
5    NaN
6    4.0
7    3.0
8    3.0
dtype: float64

## 可以实现一些string的操作！

## strip

In [50]:
idx = pd.Index([' jack', 'jill ', ' jesse ', 'frank'])

In [51]:
idx.str.strip()

Index(['jack', 'jill', 'jesse', 'frank'], dtype='object')

In [52]:
idx.str.lstrip()

Index(['jack', 'jill ', 'jesse ', 'frank'], dtype='object')

In [53]:
idx.str.rstrip()

Index([' jack', 'jill', ' jesse', 'frank'], dtype='object')

## 这些操作对于清理一些数据的字符串很有帮助，例如有空格啊，大小写啊，“_"分隔啊，什么的，可以通过这些函数把格式统一掉。

## split

In [54]:
s2 = pd.Series(['a_b_c', 'c_d_e', np.nan, 'f_g_h'])

In [55]:
s2.str.split("_")

0    [a, b, c]
1    [c, d, e]
2          NaN
3    [f, g, h]
dtype: object

## 可以获取一列的数据，通过str[]或者str.get()

In [56]:
s2.str.split("_").str[1]

0      b
1      d
2    NaN
3      g
dtype: object

In [57]:
s2.str.split("_").str.get(1)

0      b
1      d
2    NaN
3      g
dtype: object

## 把这个series变成dataframe

In [58]:
s2.str.split("_", expand = True)

,0,1,2
0,a,b,c
1,c,d,e
2,NaN,NaN,NaN
3,f,g,h


## 还可以控制split的个数

In [59]:
s2.str.split("_", expand = True, n = 1)

,0,1
0,a,b_c
1,c,d_e
2,NaN,NaN
3,f,g_h


## replace
是用正则表达式去替换。

In [60]:
s3 = pd.Series(['A', 'B', 'C', 'Aaba', 'Baca','', np.nan, 'CABA', 'dog', 'cat'])

In [61]:
s3

0       A
1       B
2       C
3    Aaba
4    Baca
5        
6     NaN
7    CABA
8     dog
9     cat
dtype: object

In [62]:
s3.str.replace('^.a|dog','xx-xx', case = False)

0          A
1          B
2          C
3    xx-xxba
4    xx-xxca
5           
6        NaN
7    xx-xxBA
8      xx-xx
9     xx-xxt
dtype: object

## 也可以单纯的代替！让regex = False即可

In [63]:
dollars = pd.Series(['12', '-$10', '$10,000'])

In [64]:
dollars.str.replace('-$','-', regex = False)

0         12
1        -10
2    $10,000
dtype: object

## fillna

In [65]:
df = pd.DataFrame([[np.nan,2,np.nan,0],
                   [3,4,np.nan,1],
                   [np.nan,np.nan,np.nan,5],
                   [np.nan,3,np.nan,4]],
                   columns=list('ABCD'))


In [66]:
df

,A,B,C,D
0,NaN,2.0,NaN,0
1,3.0,4.0,NaN,1
2,NaN,NaN,NaN,5
3,NaN,3.0,NaN,4


In [67]:
df.fillna(0)

,A,B,C,D
0,0.0,2.0,0.0,0
1,3.0,4.0,0.0,1
2,0.0,0.0,0.0,5
3,0.0,3.0,0.0,4


In [68]:
df.B.fillna(np.mean(df.B))

0    2.0
1    4.0
2    3.0
3    3.0
Name: B, dtype: float64

## ffill是拿前一个数来fill自己
pad 和ffill是一样的

In [69]:
df.fillna(method = 'ffill')

,A,B,C,D
0,NaN,2.0,NaN,0
1,3.0,4.0,NaN,1
2,3.0,4.0,NaN,5
3,3.0,3.0,NaN,4


In [70]:
df.fillna(method = 'bfill')

,A,B,C,D
0,3.0,2.0,NaN,0
1,3.0,4.0,NaN,1
2,NaN,3.0,NaN,5
3,NaN,3.0,NaN,4


## 把axis改为1，那么就会按行去填充

In [71]:
df.fillna(method = 'bfill',axis = 1)

,A,B,C,D
0,2.0,2.0,0.0,0.0
1,3.0,4.0,1.0,1.0
2,5.0,5.0,5.0,5.0
3,3.0,3.0,4.0,4.0


In [72]:
df.fillna(method = 'pad')

,A,B,C,D
0,NaN,2.0,NaN,0
1,3.0,4.0,NaN,1
2,3.0,4.0,NaN,5
3,3.0,3.0,NaN,4
